<a href="https://colab.research.google.com/github/Vikrampaswan07/Assignment-of-Business-Analyst-Intern-Jar/blob/main/Regional%20Performance%20Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Import Pandas

In [1]:
import pandas as pd

Step 2: Load Datasets

In [2]:
df_orders = pd.read_excel('/content/List_of_Orders_1.xlsx')
df_details = pd.read_excel('/content/Order_Details_1.xlsx')

In [3]:
print(df_orders.head())

  Order ID Order Date CustomerName           State       City
0  B-25601 2018-04-01       Bharat         Gujarat  Ahmedabad
1  B-25602 2018-04-01        Pearl     Maharashtra       Pune
2  B-25603 2018-04-03        Jahan  Madhya Pradesh     Bhopal
3  B-25604 2018-04-03       Divsha       Rajasthan     Jaipur
4  B-25605 2018-04-05      Kasheen     West Bengal    Kolkata


In [4]:
print(df_details.head())

  Order ID  Amount  Profit  Quantity     Category      Sub-Category
0  B-25601    1275   -1148         7    Furniture         Bookcases
1  B-25601      66     -12         5     Clothing             Stole
2  B-25601       8      -2         3     Clothing       Hankerchief
3  B-25601      80     -56         4  Electronics  Electronic Games
4  B-25602     168    -111         2  Electronics            Phones


Step 3: Marge the Datasets

In [5]:
df_merged = pd.merge(df_orders, df_details, on='Order ID')

In [6]:
print(df_merged.head())

  Order ID Order Date CustomerName        State       City  Amount  Profit  \
0  B-25601 2018-04-01       Bharat      Gujarat  Ahmedabad    1275   -1148   
1  B-25601 2018-04-01       Bharat      Gujarat  Ahmedabad      66     -12   
2  B-25601 2018-04-01       Bharat      Gujarat  Ahmedabad       8      -2   
3  B-25601 2018-04-01       Bharat      Gujarat  Ahmedabad      80     -56   
4  B-25602 2018-04-01        Pearl  Maharashtra       Pune     168    -111   

   Quantity     Category      Sub-Category  
0         7    Furniture         Bookcases  
1         5     Clothing             Stole  
2         3     Clothing       Hankerchief  
3         4  Electronics  Electronic Games  
4         2  Electronics            Phones  


Step 4: Identify Top 5 States by Order Count

In [7]:
# Get the value counts for 'State' from df_orders and grab the top 5
top_5_states_series = df_orders['State'].value_counts().head(5)

print("--- Top 5 States by Order Count ---")
print(top_5_states_series)

# Get the names of these states as a list
top_5_states_list = top_5_states_series.index.tolist()
print(f"\nAnalysis will focus on: {top_5_states_list}")

--- Top 5 States by Order Count ---
State
Madhya Pradesh    101
Maharashtra        90
Rajasthan          32
Gujarat            27
Punjab             25
Name: count, dtype: int64

Analysis will focus on: ['Madhya Pradesh', 'Maharashtra', 'Rajasthan', 'Gujarat', 'Punjab']


Step 5: Create a Full Regional Analysis Table

In [8]:
# Group the merged data by 'State' and aggregate all necessary metrics
full_regional_analysis = df_merged.groupby('State').agg(
    Total_Sales=('Amount', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Total_Orders=('Order ID', 'nunique')
).reset_index()

# Calculate Average Profit per Order for all states
full_regional_analysis['Avg_Profit_per_Order'] = full_regional_analysis['Total_Profit'] / full_regional_analysis['Total_Orders']

print("\n--- Full Regional Analysis (Sample) ---")
print(full_regional_analysis.head())


--- Full Regional Analysis (Sample) ---
            State  Total_Sales  Total_Profit  Total_Orders  \
0  Andhra Pradesh        13256          -496            15   
1           Bihar        12943          -321            16   
2           Delhi        22531          2987            22   
3             Goa         6705           370            14   
4         Gujarat        21058           465            27   

   Avg_Profit_per_Order  
0            -33.066667  
1            -20.062500  
2            135.772727  
3             26.428571  
4             17.222222  


Step 6: Report on the Top 5 States

In [9]:
# Filter the full analysis to only include the top 5 states
# We set 'State' as the index to filter using .loc
top_5_report = full_regional_analysis.set_index('State').loc[top_5_states_list]

# Sort by Total Sales for a clean report
top_5_report_sorted = top_5_report.sort_values(by='Total_Sales', ascending=False)

print("\n--- Performance of Top 5 States ---")
# Print the columns requested by the prompt, with formatting
print(top_5_report_sorted[['Total_Sales', 'Avg_Profit_per_Order']].to_string(float_format='${:,.2f}'.format))



--- Performance of Top 5 States ---
                Total_Sales  Avg_Profit_per_Order
State                                            
Madhya Pradesh       105140                $54.96
Maharashtra           95348                $68.62
Rajasthan             21149                $39.28
Gujarat               21058                $17.22
Punjab                16786               $-24.36


Step 7: Highlight Regional Disparities (Analysis)

In [10]:
print("\n--- Regional Disparity Analysis (Top 5) ---")
print(f"Based on the table above, we can see clear disparities:\n")

# Get data for analysis
maharashtra = top_5_report.loc['Maharashtra']
mp = top_5_report.loc['Madhya Pradesh']

print(f"1. High Sales, High Profit: 'Maharashtra' is the top performer in both sales (${maharashtra['Total_Sales']:,.2f}) and profitability (${maharashtra['Avg_Profit_per_Order']:,.2f} per order).")
print(f"2. High Sales, Low Profit: 'Madhya Pradesh' has the 2nd highest sales (${mp['Total_Sales']:,.2f}) but a significantly lower average profit per order (${mp['Avg_Profit_per_Order']:,.2f}).")
print("   - This suggests that while sales volume is high in MP, the orders are less profitable, possibly due to discounts, product mix, or higher operational costs.")



--- Regional Disparity Analysis (Top 5) ---
Based on the table above, we can see clear disparities:

1. High Sales, High Profit: 'Maharashtra' is the top performer in both sales ($95,348.00) and profitability ($68.62 per order).
2. High Sales, Low Profit: 'Madhya Pradesh' has the 2nd highest sales ($105,140.00) but a significantly lower average profit per order ($54.96).
   - This suggests that while sales volume is high in MP, the orders are less profitable, possibly due to discounts, product mix, or higher operational costs.


Step 8: Identify Regions for Improvement

In [11]:
# Sort the full analysis by 'Avg_Profit_per_Order' (ascending) to find the bottom 5
bottom_profit_states = full_regional_analysis.sort_values(by='Avg_Profit_per_Order').head(5)

print("\n--- Top 5 States to Prioritize for Improvement (Lowest Avg. Profit) ---")
print(bottom_profit_states[['State', 'Total_Sales', 'Avg_Profit_per_Order']].to_string(index=False, float_format='${:,.2f}'.format))



--- Top 5 States to Prioritize for Improvement (Lowest Avg. Profit) ---
            State  Total_Sales  Avg_Profit_per_Order
       Tamil Nadu         6087              $-277.00
   Andhra Pradesh        13256               $-33.07
           Punjab        16786               $-24.36
            Bihar        12943               $-20.06
Jammu and Kashmir        10829                 $0.57


Step 9: Suggest Specific Cities for Improvement

In [12]:
# Get the name of the worst-performing state
worst_state = bottom_profit_states.iloc[0]['State']

print(f"\n--- Drilling Down: City-Level Analysis for {worst_state} ---")

# Filter the merged data for just that state
worst_state_cities = df_merged[df_merged['State'] == worst_state]

# Group by City and calculate metrics
worst_city_analysis = worst_state_cities.groupby('City').agg(
    Total_Sales=('Amount', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Total_Orders=('Order ID', 'nunique')
)
worst_city_analysis['Avg_Profit_per_Order'] = worst_city_analysis['Total_Profit'] / worst_city_analysis['Total_Orders']

# Sort by Avg_Profit_per_Order to find the problem city
print(worst_city_analysis.sort_values(by='Avg_Profit_per_Order').to_string(float_format='${:,.2f}'.format))

# Get the worst city
worst_city = worst_city_analysis.sort_values(by='Avg_Profit_per_Order').iloc[0]

print(f"\nSuggestion: 'Bihar' (and specifically 'Patna') should be prioritized for improvement.")
print(f"Patna has an average profit of {worst_city['Avg_Profit_per_Order']:,.2f} per order, indicating a significant problem with profitability in this region.")



--- Drilling Down: City-Level Analysis for Tamil Nadu ---
         Total_Sales  Total_Profit  Total_Orders  Avg_Profit_per_Order
City                                                                  
Chennai         6087         -2216             8              $-277.00

Suggestion: 'Bihar' (and specifically 'Patna') should be prioritized for improvement.
Patna has an average profit of -277.00 per order, indicating a significant problem with profitability in this region.
